In [2]:
import pandas as pd
import numpy as np
import requests
df = pd.read_csv('成都商铺数据.csv',error_bad_lines=False)

In [6]:
lst_lpname = df['loupanmingcheng'].value_counts().index.tolist()

lst_lploc = df['loupandizhi'].value_counts().index.tolist()

In [10]:
city = '成都'
dict_location = {}
My_key = 'f654d644aeae0e1da847d7ca408f4370'
url_pre = 'https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}'
counter = 0
for i in range(len(lst_lpname)):
    location = lst_lpname[i]
    url = url_pre.format(My_key, location, city)
    res = requests.get(url).json()
    if res['status'] == '0':
        location = lst_lploc[i]
        url = url_pre.format(My_key, location, city)
        res = requests.get(url).json()
        if res['status'] == '0':
            dict_location[lst_lpname[i]] = np.nan
            counter +=1
        else:
            try: 
                dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                if counter % 250 == 0:
                    print('one batch(250) processed')
            except:
                dict_location[lst_lpname[i]] = np.nan
            
    else:
        try: 
            dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
            if counter % 250 == 0:
                print('one batch(250) processed')
        except:
            try:
                location = lst_lploc[i]
                url = url_pre.format(My_key, location, city)
                res = requests.get(url).json()
                if res['status'] == '0':
                    dict_location[lst_lpname[i]] = np.nan
                    counter +=1
                else:
                    try: 
                        dict_location[lst_lpname[i]] = res['geocodes'][0]['location']
                        if counter % 250 == 0:
                            print('one batch(250) processed')
                    except:
                        dict_location[lst_lpname[i]] = np.nan
            except:
                dict_location[lst_lpname[i]] = np.nan
                
            
        counter +=1

one batch(250) processed


In [11]:
df['location_coords'] = np.nan

counter = 0
for i in lst_lpname:
    counter += 1
    df_temp = df[df['loupanmingcheng'] == i]
    df_temp_index = df_temp.index.tolist()
    df.iloc[df_temp_index,22] = dict_location[i]
    if counter %50 == 0:
        print('A batch(50) is done')

A batch(50) is done
A batch(50) is done
A batch(50) is done


In [12]:
df.head()

,zhuanrangfei,rental_price,pay_method,jianzhumianji,shiyongmianji,shengyuzuqi,zhuanrangfangshi,floor,zhuangxiu,building_type,...,loupandizhi,miankuan,jinshen,cenggao,elvator_number,service_price,function_list,release_date,url,location_coords
0,6万元,5000元/月,押一付二,38㎡,NaN,2个月,NaN,第1层(共1层),精装修\r\n,临街门面,...,\r\n 四川省成都市...,3.50米,6.00米,5.50米,NaN,租金已包含,"暖气,中央空调,下水,排污,外摆区",2019/8/4 13:20:41,https://cd.shop.fang.com/zu/1_52957904.html,"104.078899,30.652903"
1,NaN,8000元/月,季付,320㎡,NaN,NaN,NaN,第2层(共3层),精装修\r\n,临街门面,...,\r\n 龙湖金楠天街,NaN,NaN,NaN,NaN,NaN,"网络,暖气,停车位,客梯,货梯",2019/8/27 15:06:05,https://cd.shop.fang.com/zu/1_52961200.html,"104.069611,30.653686"
2,NaN,17000元/月,半年付,100㎡,NaN,NaN,NaN,第1层(共1层),简装修,商业街商铺,...,首创光和城,20.00米,40.00米,5.10米,NaN,租金已包含,"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气,外摆区",2019/6/15 18:13:59,https://cd.shop.fang.com/zu/3_207938226.html,"104.115777,30.711008"
3,NaN,56000元/月,面议,800㎡,700㎡,NaN,NaN,第2层(共3层),简装修,临街门面,...,锦江区，青羊区，武侯区，成华区，金牛区 郫都区，新,40.00米,20.00米,4.50米,NaN,5.00元/㎡·月,"网络,暖气,停车位,客梯,货梯,扶梯,中央空调,上水,下水,可明火,排烟,排污,燃气",2019/6/23 10:21:20,https://cd.shop.fang.com/zu/3_208016008.html,"104.080150,30.386622"
4,NaN,10000元/月,租金已包含,暂无,NaN,NaN,NaN,精装修,其他,否,...,5.00米,4.00米,3.00米,EGO潮流广场,16649.99㎡,24㎡,NaN,2019/8/2 22:48:20,https://cd.shop.fang.com/zu/3_208453615.html,"104.061648,30.573185"


In [13]:
df.to_csv('成都商铺地址数据.csv', encoding = 'utf-8-sig',index = False)

In [14]:
len(lst_lploc)

166

In [15]:
df.shape

(484, 23)